Nome: Fabio Grassiotto  
RA: 890441

# Aula 9_10 - Implementar a ReAct  com LLaMa 3 70B (groq)

- Testar no dataset do IIRC - 50 primeiras perguntas com resposta (test_questions.json em anexo)  
- Usar o prompt do LLaMAIndex: https://github.com/run-llama/llama_index/blob/a87b63fce3cc3d24dc71ae170a8d431440025565/llama_index/agent/react/prompts.py  
- Salvar as respostas finais das 50 perguntas no JSON para exercício futuro de avaliação  
- Instruir o modelo a seguir a sequência Thougth, Action, Input, Observation (a observação não é do próprio modelo, mas resultado da busca)  
- É necessário usar o parâmetro stop_sequence="Observation:", para o modelo parar de gerar texto e esperar o retorno da busca. Implementem o código da busca e retornem os top-k documentos pro modelo (sugestão: k=5).  
- Instruir o modelo agir passo-a-passo (decomposição da pergunta).  
- Podem usar o LangChain, LLaMAindex ou outro framework. Ou implementar na mão.  
- Usar a busca como ferramenta  
- Usar o BM25 como buscador (repetir indexação do exercício passado)  
- Usar a indexação do Visconde: https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipyn  


## Setup Environment
### Install packages

In [1]:
%%capture
%pip install -q torch
%pip install groq
%pip install llama-index-llms-groq
%pip install langchain
%pip install langchain-groq
%pip install rank_bm25
%pip install bs4

### Imports

In [2]:
import os
import sys
import json
import torch
from collections import Counter
import string
import re
from bs4 import BeautifulSoup
from rank_bm25 import BM25Okapi
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm

import warnings
warnings.simplefilter('ignore')

### Collab setup

In [3]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_9_10"
    os.chdir(project_folder)
    !ls -la

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Globals

In [4]:
NUM_QUESTIONS = 50
MODEL_NAME="llama3-70b-8192"

## IIRC Dataset

In [5]:
if not os.path.exists("dataset/context_articles.json"):
    !wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
    !tar -xzf context_articles.tar.gz --directory dataset
    !rm context_articles.tar.gz

In [6]:
test_dataset  = json.load(open('dataset/test_questions.json', 'r'))
articles = json.load(open('dataset/context_articles.json', 'r'))

In [7]:
test_dataset[0]

{'answer': {'type': 'span',
  'answer_spans': [{'text': 'sky and thunder god',
    'passage': 'zeus',
    'type': 'answer',
    'start': 83,
    'end': 102}]},
 'question': 'What is Zeus know for in Greek mythology?',
 'context': [{'text': 'he Palici the sons of Zeus',
   'passage': 'main',
   'indices': [684, 710]},
  {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]},
  {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion',
   'passage': 'Zeus',
   'indices': [0, 110]}],
 'question_links': ['Greek mythology', 'Zeus'],
 'title': 'Palici'}

### Load and parse

In [8]:
# Grab the first 50 questions with an answer
def grab_documents():
  questions_found = []
  num_questions_found = 0
  documents = []
  all_titles = []

  for item in test_dataset:
    
    question = item['question']
    answer = item['answer']
    answer_type = answer['type']

    if answer_type == 'binary' or answer_type == 'value':
      final_answer = answer['answer_value']
    elif answer_type == 'span':
      final_answer = answer['answer_spans'][0]['text']
    elif answer_type == 'none':
      final_answer = 'none'
    else:
      final_answer = 'An error perhaps, bad type'
      print(answer_type)

    if (final_answer == 'none'):
      # Skip this one.
      continue
    else:
      
      # Thats a good question, grab the document and title associated with it
      # from the question_links element in the test set.

      for link in item["question_links"]:
        # Context
        context_list = item['context']
        for context in context_list:
          text = context['text']
          # clean up html
          soup = BeautifulSoup(text, 'html.parser')
          clean_text = soup.get_text()

          documents.append({
            "title": item['title'],
            "content": clean_text
          })
          all_titles.append(item['title'].lower())

        # Articles
        if link.lower() in articles:
          # clean up html
          soup = BeautifulSoup(articles[link.lower()], 'html.parser')
          clean_text = soup.get_text()

          documents.append({
            "title": link,
            "content": clean_text
            })
          all_titles.append(link.lower())

      questions_found.append({"Question": question, "Answer": final_answer})
      num_questions_found += 1
      if (num_questions_found == NUM_QUESTIONS):
        # found our questions
        break

  return questions_found, documents

questions_to_ask, documents = grab_documents()

### Inspecting data

In [9]:
len(questions_to_ask)

50

In [10]:
len(documents)

275

In [11]:
questions_to_ask[0]

{'Question': 'What is Zeus know for in Greek mythology?',
 'Answer': 'sky and thunder god'}

In [12]:
documents[:5]

[{'title': 'Palici', 'content': 'he Palici the sons of Zeus'},
 {'title': 'Palici', 'content': 'in Greek mythology'},
 {'title': 'Palici',
  'content': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion'},
 {'title': 'Greek mythology',
  'content': 'Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore. These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks\' own cult and ritual practices. Modern scholars study the myths in an attempt to shed light on the religious and political institutions of ancient Greece and its civilization, and to gain understanding of the nature of myth-making itself.\n\nThe Greek myths were initially propagated in an oral-poetic tradition most likely by Minoan and Mycenaean singers starting in the 18th century

## BM25 Indexing

### Simple test using Rank_BM25

In [13]:
def search_top5(documents, query):
    corpus = [d.get('content', 'default_value') for d in documents]
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    tokenized_query = query.split(" ")
    result_raw = bm25.get_top_n(tokenized_query, corpus, n=5)
    
    # Reduce the number of tokens in the result to fit the Groq API.
    result = []
    for res in result_raw:
        str = res[:512]
        result.append(str)
    return result

search_top5(documents, questions_to_ask[30]['Question'])

['The 2004 National League Division Series (NLDS), the opening round of the 2004 National League playoffs, began on Tuesday, October 5, and ended on Monday, October 11, with the champions of the three NL divisions—along with a "wild card" team—participating in two best-of-five series. They were:\n\n- (1) St. Louis Cardinals (Central Division champion, 105–57) vs. (3) Los Angeles Dodgers (Western Division champion, 93–69): Cardinals win series, 3–1.\n- (2) Atlanta Braves (Eastern Division champion, 96–66) vs. (4)',
 "The Rochester Rhinos are an American professional soccer team based in Rochester, New York, United States. Founded in 1996, as the Rochester Raging Rhinos, the team formerly played in the United Soccer League in the second tier of the United States soccer league system, and is currently on hiatus. They changed their name to Rochester Rhinos to start the 2008 season. The club is the only non-MLS team to have won the U.S. Open Cup since the league's formation.\n\nOn November 

## Langchain Groq API

In [14]:
def load_groq_key():
    try:
        # Open and read the entire content of the file
        with open("groq-key.txt", 'r') as file:
            contents = file.read()
        
        return contents
    
    except FileNotFoundError:
        print(f"The file does not exist.")
        return None
    except Exception as e:
        # Handle other potential exceptions (e.g., permission errors)
        print(f"An error occurred while reading the file: {str(e)}")
        return None
    
chat = ChatGroq(temperature=0, groq_api_key=load_groq_key(), model_name=MODEL_NAME)

In [15]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:\n\n1. **Interactive Systems**: In applications like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, creating a more natural and engaging user experience. This is particularly important in customer-facing applications where delayed responses can lead to frustration and abandonment.\n2. **Real-time Decision Making**: In areas like finance, healthcare, and cybersecurity, timely decision-making is critical. Low-latency LLMs can quickly analyze vast amounts of data, enabling swift identification of patterns, anomalies, and insights, which inform timely decisions.\n3. **Live Content Generation**: Low-latency LLMs are essential for generating content in real-time, such as live subtitles, closed captions, or automated summarization of

## ReAct Agent Implementation

### Prompt from Langchain (https://smith.langchain.com/hub/hwchase17/react-chat) - slightly modified to remove chat history.

In [26]:
LANGCHAIN_CHAT_SYSTEM_HEADER = """\

Assistant is a large language model trained by Meta AI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------
Assistant has access to the following tools:
{tools}
To use a tool, please use the following format:
```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```
When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```
Begin!
New input: {input}
{agent_scratchpad}
"""

### Agent tool definition

In [17]:
@tool
def get_top5(question: str) -> list[str]:
    """Returns a list of strings with texts to help solve a question."""
    str_list = search_top5(documents, question)
    return str_list

In [18]:
print(get_top5.name)
print(get_top5.description)
print(get_top5.args)

get_top5
get_top5(question: str) -> list[str] - Returns a list of strings with texts to help solve a question.
{'question': {'title': 'Question', 'type': 'string'}}


### Defining the Agent Executor

In [27]:
prompt = PromptTemplate.from_template(LANGCHAIN_CHAT_SYSTEM_HEADER)
llm = ChatGroq(temperature=0,
               model_name=MODEL_NAME,
                      api_key=load_groq_key())
tools = [get_top5]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)


In [28]:
agent_executor.invoke({"input": "Question: " + questions_to_ask[1]['Question']})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_top5
Action Input: How long had the First World War been over when Messe was named aide-de-camp?['World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918. Contemporaneously described as "the war to end all wars", it led to the mobilisation of more than 70\xa0million military personnel, including 60 million Europeans, making it one of the largest wars in history. It is also one of the deadliest conflicts', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918. Contemporaneously described a

{'input': 'Question: How long had the First World War been over when Messe was named aide-de-camp?',
 'output': 'Wilhelm Messe was named aide-de-camp in 1934. The First World War ended in 1918, so it had been over for 16 years when Messe was named aide-de-camp.'}

## Execution Run

### Evaluation functions

In [21]:
def normalize_answer(s):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  Lower text and remove punctuation, articles and extra whitespace.
  """

  def remove_articles(text):
      return re.sub(r"\b(a|an|the)\b", " ", text)

  def white_space_fix(text):
      return " ".join(text.split())

  def remove_punc(text):
      exclude = set(string.punctuation)
      return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
      return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  """
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
      return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

### Analysing the output for 50 questions

In [22]:
# Create dataframe with question, answer, Llm answer and scores
import pandas as pd
df_results = pd.DataFrame(columns=['question', 'answer', 'LLM answer', 'F1', 'Exact Match'])

for item in tqdm(questions_to_ask):
    question = item.get('Question')
    answer = normalize_answer(item.get('Answer'))

    # RAG context using search
    agent_answer = agent_executor.invoke({"input": "Question: " + question})
    llm_answer = normalize_answer(agent_answer['output'])
    f1_score = token_f1_score(llm_answer, answer)
    e_match_score = 1 if llm_answer == answer else 0
    
    row = pd.Series([question, answer, llm_answer, f1_score, e_match_score], index=df_results.columns)
    df_results = pd.concat([df_results, pd.DataFrame([row])], ignore_index=True)

  0%|          | 0/50 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes

Action: get_top5

Action Input: What is Zeus know for in Greek mythology?
['Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus. His name is cognate with the first element of his Roman equivalent Jupiter. His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor.\n\nZeus is the child of Cronus and Rhea, the youngest of his siblings to be born, though sometimes reckoned the eldest as the others', "Greek mythology is the body of myths originally told by the ancient Greeks and a genre of Ancient Greek folklore. These stories concern the origin and the nature of the world, the lives and activities of deities, heroes, and mythological creatures, and the origins and significance of the ancient Greeks' own cult and ritual pr

  2%|▏         | 1/50 [00:06<05:21,  6.56s/it]

Do I need to use a tool? No

Final Answer: Zeus is the king of gods in Greek mythology.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes

Action: get_top5

Action Input: How long had the First World War been over when Messe was named aide-de-camp?
['World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a global war originating in Europe that lasted from 28 July 1914 to 11 November 1918. Contemporaneously described as "the war to end all wars", it led to the mobilisation of more than 70\xa0million military personnel, including 60 million Europeans, making it one of the largest wars in history. It is also one of the deadliest conflicts', 'World War I (often abbreviated as WWI or WW1), also known as the First World War, the Great War, the Seminal Catastrophe, and initially in North America as the European War, was a glo

  4%|▍         | 2/50 [00:09<03:32,  4.42s/it]

...

Do I need to use a tool? No

Final Answer: No, he was not alive.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes

Action: get_top5

Action Input: Which of the battles Sherman fought during a winter campaign had higher Union casualty rate?
['Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', 'Sherman returned to serve under General Ulysses S. Grant in the winter of 1862 during the battles of forts Henry and Donelson.', "World War\xa0II (often abbreviated to WWII or WW2), also known as the Second World War, was a global war that lasted from 1939 to 1945. The vast majority of the world's countries—including all the great powers—eventually formed two opposing military alliances: the Allies and the Axis. A state of total war emerged, directly involving more than 100 million people from more than 30 countries. The major participants threw their entire economic, indus

 98%|█████████▊| 49/50 [11:09<00:14, 14.10s/it]

Do I need to use a tool? No

Final Answer: Battle of Fort Donelson.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes

Action: get_top5

Action Input: Which country did some of the tribes escaped to after their deportation from northern Mexico?
['Those who escaped deportation were dispersed across northern Mexico and some even settled across the international border in southern Arizona', 'In the Battle of Bautzen (20–21 May 1813) a combined Russian–Prussian army, that was massively outnumbered, was pushed back by Napoleon I of France but escaped destruction, some sources claiming that Michel Ney failed to block their retreat. The Prussians under Count Gebhard von Blücher and Russians under Prince Peter Wittgenstein, retreating after their defeat at Lützen were attacked by French forces under Napoleon.\n\nPrelude.The Prusso-Russian army was in a full retreat following their defeat at the Battle', 'Costa Rica (, ; ; literally "Rich Coast"), 

100%|██████████| 50/50 [11:25<00:00, 13.71s/it]

Thought: Do I need to use a tool? No

Final Answer: United States

> Finished chain.


### Results

In [23]:
df_results

,question,answer,LLM answer,F1,Exact Match
0,What is Zeus know for in Greek mythology?,sky and thunder god,zeus is king of gods in greek mythology,0,0
1,How long had the First World War been over whe...,5,messe was named aidedecamp in 1934,0,0
2,How old was Messe when the First World War sta...,30,messe was 31 years old,0,0
3,How long had Angela Scoular been acting profes...,2,angela scoular acted professionally for 4 years,0,0
4,What is the capacity of the stadium where Brun...,26688,hawthorns has capacity of 26688,0.333333,0
5,Which stadium where Brunt played can hold more...,white hart lane,white hart lane,1.0,1
6,In which country was Wilhelm Müller born?,germany,germany,1.0,1
7,Which battle Wilhelm Müller fought in while in...,kulm,battle of lützen had highest casualty,0,0
8,How much time had passed between Wilhelm's ret...,9,wilhelms son friedrich was born in 1823,0,0
9,Which of Wilhelm's direct male descendants liv...,friedrich max müller,wilhelm iis son wilhelm,0,0


In [24]:
df_results.to_csv("results/react_results.csv")

In [25]:
print("Metrics:")
print(f"F1 score: Avg: {df_results['F1'].mean():.2f}. Std: {df_results['F1'].std():.2f}")
print(f"Exact Match score: Avg: {df_results['Exact Match'].mean():.2f}. Std: {df_results['Exact Match'].std():.2f}")

Metrics:
F1 score: Avg: 0.29. Std: 0.39
Exact Match score: Avg: 0.18. Std: 0.39
